# Support Vector Machines

SVM sind ein flexibles und mächtiges ML-Modell. Man kann mit ihnen sowohl lineare als auch nicht-lineare Klassifikationsaufgaben, Regression und die Erkennung von Ausfreissern bewältigen.  
## Large Margin Klassifikation
Probiert eine möglichst breite Strasse zwischen die Kategorien zu bauen um diese zu separieren.
Diese Strasse wird durch die Punkte am Rande der Strasse gestützt/determiniert. Diese Punkte heissen support vectors (Stützvektoren).  

**Skalierung:**  
SVMs sind sehr empfindlich auf Skalierungen. Mit Skalierten Merkmalen können sie viel präzisere Vorhersagen treffen.

### Hard Margin Klassifikation
Hard Margin probiert eine möglichst breite Strasse zwischen den Kategorien zu erstellen.  
Die Hard Margin Klassifikation hat zwei Nachteile:
* Funktioniert nur, wenn die Daten linear separierbar sind
* Anfällig für Ausreisser (Margin-Verletzungen, dies sind Punkte, welche auf der Strasse liegen)

### Soft Margin Klassifikation
Findet im gegensatz zur Hard-Margin-Klassifikation eine gute Balance zwischen der Strassenbreite und der Anzahl von Margin-Verletzungen.  

**Hyperparameter C:**  
Dieser Hyperparameter steuert die Balance zwischen Strassenbreite und Margin-Verletzungen.
* kleiner Wert für C:
    * breite Strasse
    * mehr Margin-Verletzungen
* grosser Wert für C:
    * schmale Strasse
    * weniger Margin-Verletzungen

**Overfitting:**  
Wenn SVM overfittet, kann man es durch Senken von **C** regularisieren.

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [3]:
iris = load_iris()
X = iris['data'][:, (2,3)]
y = (iris['target'] == 2).astype(np.float64)

In [4]:
svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=1, loss='hinge'))
])

In [5]:
svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [8]:
svm_clf.predict([[5.5, 1.7]])

array([1.])